In [1]:
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from BERT_explainability.modules.BERT.BertForSequenceClassification import BertForSequenceClassification
from transformers import AutoTokenizer

from captum.attr import visualization
import torch

In [2]:
# Bert로 model 불러오기 

model = BertForSequenceClassification.from_pretrained('klue-bert-epoch-32').to("cuda:0")
model.eval()
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [3]:
# encode a sentence
text_batch = ["야 너희 립스틱 이쁘다 엄마가 성적 3등 올랐다고 사준거야 나 오늘 남자친구랑 데이트 하는데 좀 빌려줘라 안 돼 저번에도 너 빌려 가서 망가트렸잖아 너 많이 컸다 이제 말대꾸도 하냐 그게 아니라 엄마가 백화점 가서 특별히 사 준 건데 그러니까 더 좋네 남자친구한테 잘 보이고 오늘 내가 빌려 갈테니까 그런 줄 알아 어디 가서 꼬바르지 말고 알아서 잘 해라 안 되는데"]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda:0")
attention_mask = encoding['attention_mask'].to("cuda:0")

In [37]:
# initialize the explanations generator
explanations = Generator(model)